In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import pytz

import psycopg2
import plotly.express as px
import datetime

def init_connection():
    return psycopg2.connect(**st.secrets["postgres"])
conn = init_connection()

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def load_data():
    df = pd.read_sql_query('select * from "atendimentos"',con=conn)
    return df

data = load_data()
data.head(2)

,id,tipo_senha,numeracao,codigo_senha,data_emissao,guiche,data_atendimento
0,24,SP,7,SP007,2022-09-06 16:41:47.444,1.0,2022-09-06 17:07:19.874
1,25,SP,8,SP008,2022-09-06 11:43:22.651,1.0,2022-09-06 12:07:26.636


In [8]:
data = data[data['numeracao']==20]


In [33]:
def timedelta_to_string(x):
    if pd.isnull(x):
        return '-'
    total_sec = x.total_seconds()
    hours = int(total_sec / 3600)
    minutes = int(total_sec / 60) % 60
    sec = int(total_sec - minutes*60 - hours*60*60)
    time_string = str(hours).zfill(2) + ":" + str(minutes).zfill(2) + ":" + str(sec).zfill(2)
    return time_string

In [9]:
data_show = data[["tipo_senha","numeracao","data_emissao","data_atendimento","guiche"]]
data_show = data_show.sort_values(by = "data_emissao")
data_show["guiche"] = data_show["guiche"].astype('string').fillna('---').str[:-2]
data_show["tempo_atendimento"] = data_show["data_atendimento"] - data_show["data_emissao"]
tempo_medio_por_tipo = data_show[['tipo_senha','tempo_atendimento']].groupby('tipo_senha').mean()
data_show

## Converter
data_show["data_emissao"] = data_show["data_emissao"].dt.strftime('%Y-%m-%d, %H:%m:%S')
data_show["data_atendimento"] = data_show["data_atendimento"].dt.strftime('%Y-%m-%d, %H:%m:%S')
data_show["tempo_atendimento"] = data_show["tempo_atendimento"].apply(lambda x : timedelta_to_string(x))


data_show.rename(columns = {
    "tipo_senha":"Tipo"
    ,"numeracao":"Numeração"
    , "data_emissao":"Data de Emissão"
    , "data_atendimento":"Data de Atendimento"
    , "guiche":"Guichê"
    , "tempo_atendimento":"Tempo de Espera"
    }, inplace=True)

data_show.head(3)


,Tipo,Numeração,Data de Emissão,Data de Atendimento,Guichê,Tempo de Espera


In [12]:
len(tempo_medio_por_tipo)

0

,Tipo,Tempo Médio


In [10]:
tempo_medio_por_tipo
tempo_medio_por_tipo["tempo_atendimento"] = tempo_medio_por_tipo["tempo_atendimento"].apply(lambda x : timedelta_to_string(x))
tempo_medio_por_tipo = tempo_medio_por_tipo.reset_index()
tempo_medio_por_tipo.rename(columns = {
    "tipo_senha":"Tipo"
    , "tempo_atendimento":"Tempo Médio"
    }, inplace=True)


KeyError: 'tempo_atendimento'

In [99]:
tempo_medio_por_tipo

,Tipo,Tempo Médio
0,SE,00:27:31
1,SG,00:16:12
2,SP,00:19:04


In [62]:
tempo_medio_por_tipo["tempo_atendimento"]

tipo_senha
SE   0 days 00:27:31.550492222
SG      0 days 00:16:12.624682
SP   0 days 00:19:04.554344761
Name: tempo_atendimento, dtype: timedelta64[ns]

In [63]:
tempo_medio_por_tipo["tempo_atendimento"].apply(lambda x : timedelta_to_string(x))

tipo_senha
SE    00:27:31
SG    00:16:12
SP    00:19:04
Name: tempo_atendimento, dtype: object

In [58]:
teste.total_seconds()

1532.43

In [ ]:
from dateutil.relativedelta import relativedelta




In [ ]:
data.data_emissao[0].month

In [ ]:
data['hora_emissao'] = data['data_emissao'].dt.hour
data['hora_atendimento'] = data['data_atendimento'].dt.hour

In [ ]:
data.rename(columns = {"hora_emissao":"Hora Emissao", "id":"Count"}).groupby(by=["Hora Emissao"]).count()[["Count"]].sort_values(by="Count")

In [ ]:
data.groupby(by=["tipo_senha"]).count()[["id"]].sort_values(by="id")

In [ ]:
data.groupby('tipo_senha').size()

In [ ]:
tipos = data['tipo_senha'].unique()
for tipo in tipos:
    cond1 = ~data["data_atendimento"].isna()
    cond2 = data["tipo_senha"] == tipo
    senha_por_tipo = int(len(data[cond1 & cond2]))
    print(f"Senhas Atendidas {tipo}: {senha_por_tipo}")